In [1]:
%run 'naoqi/naoqi.py'
%run 'naoqi/almath.py'

In [2]:
import sys
import motion
import time
import math

In [3]:
robotIP = "169.254.226.148" # A changer selon les cas

In [5]:
# Ouvre les connexions
motionProxy = ALProxy("ALMotion", robotIP, 9559) 
postureProxy = ALProxy("ALRobotPosture", robotIP, 9559)

In [6]:
def StiffnessOn(proxy):
  #We use the "Body" name to signify the collection of all joints
  pNames = "Body"
  pStiffnessLists = 1.0
  pTimeLists = 1.0
  proxy.stiffnessInterpolation(pNames, pStiffnessLists, pTimeLists)

def StiffnessOff(proxy):
  #We use the "Body" name to signify the collection of all joints
  pNames = "Body"
  pStiffnessLists = 0
  pTimeLists = 1.0
  proxy.stiffnessInterpolation(pNames, pStiffnessLists, pTimeLists)


In [9]:
StiffnessOff(motionProxy)

In [10]:
StiffnessOn(motionProxy)

In [11]:
postureProxy.goToPosture("StandInit", 0.5)

True

In [194]:
def cartesian_control():
    # Setting posture StandInit before hand is mandatory
    effector   = "LArm"
    space      = motion.FRAME_ROBOT
    path       = [ # Chemin représenté comme un ligne de points
     [0.0, +0.025, +0.00, 0.0, 0.0, 0.0],        # point 1
     [0.025, +0.0, +0.00, 0.0, 0.0, 0.0],        # point 2
     [0.0, -0.025, +0.00, 0.0, 0.0, 0.0],        # point 3
     [-0.025, +0.0, -0.00, 0.0, 0.0, 0.0]]#,        # point 4
   #  [0.0, -0.05, +0.00, 0.0, 0.0, 0.0],        # point 5
   #  [0.0, +0.05, +0.00, 0.0, 0.0, 0.0]] 
    axisMask = 7
    # Moment auxquel on doit passer aux points considérés
    times = [1, 2.0, 3.0, 4.0]#, 4.0, 4.5] # seconds
    isAbsolute = False
    
    motionProxy.positionInterpolation(effector, space,
            path,axisMask, times, isAbsolute)

In [ ]:
cartesian_control()

In [195]:
pi = math.pi

def radian(a):
    return a * pi / 180

def angles():
    # Exemples de positionnement avec les angles
    motionProxy.setStiffnesses("LArm", 1.0)

    # Example showing how to set angles, using a fraction of max speed
    names  = ["LShoulderPitch", "LElbowYaw"]
    angles  = [radian(0), radian(0)]
    fractionMaxSpeed  = 0.1
    motionProxy.setAngles(names, angles, fractionMaxSpeed)

    time.sleep(3.0)
    #motionProxy.setStiffnesses("LArm", 0.0)


In [189]:
angles()

In [197]:
def angle2():
    # Example showing multiple trajectories
    names  = ["LShoulderPitch","LElbowRoll","LElbowYaw"]
    # Each joint can have lists of different lengths, but the number of
    # angles and the number of times must be the same for each joint.
    angleLists  = [ [30*TO_RAD,0],
                    [-50.0*TO_RAD, 0.0],
                   [-30.0*TO_RAD, 30.0*TO_RAD, 0.0]]
    timeLists   = [[1.0,6.0],[2.0, 3.0], [ 2.0, 3.0, 4.0]]
    isAbsolute  = True
    motionProxy.angleInterpolation(names, angleLists, timeLists, isAbsolute)
    

In [198]:
angle2()